# В этом ноутбуке находятся бенчмарки для сравнения CBS и ECBS на трех картах: random с 10% препятствий, лабиринт и реальная карта с малым числом препятствий.

In [ ]:
from stats import run_algo
from typing import List
from tqdm.notebook import tqdm
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

In [10]:
import os
filename = 'result2.txt'

def read_expanded(filename=filename):
    with open(filename, 'r') as path_file:
        hl_expanded, ll_expanded = list(map(int, path_file.read().split('\n')[0].split(' ')[1:]))
        return hl_expanded, ll_expanded

def run(alg_name, map_name, scen_name, num_agents, num_repetitions, w=None):
    hl_expanded = 0
    ll_expanded = 0
    count_done = 0
    for i in tqdm(range(num_repetitions)):
        done = run_algo(alg=alg_name, dest_file=filename,
                        map_path=os.path.join(os.curdir, 'data', 'maps', 'mapf', map_name),
                        scen_path=os.path.join(os.curdir, 'data', 'scens', 'mapf', scen_name),
                        tasks_count=num_agents,
                        test_num=i, w=w)
        count_done += int(done)
        if done:
            hl, ll = read_expanded(filename)
            hl_expanded += hl * int(done)
            ll_expanded += ll * int(done)
    if count_done == 0:
        return (0, 0, 0)
    return count_done / num_repetitions, hl_expanded / count_done, ll_expanded / count_done

In [11]:
from collections import defaultdict
def parse_path_file(path_file_content):
    agents_path = defaultdict(list)
    agent = None
    lines = path_file_content.split('\n')[1:]
    for line in lines:
        if line == '':
            break
        nums = list(map(int, line.split(' ')))
        if len(nums) == 1:
            agent = int(nums[0])
        else:
            x, y, time = nums
            agents_path[agent].append((x, y, time))
    return agents_path

def read_paths(filename=filename):
    with open(filename, 'r') as path_file:
        path_file_content = path_file.read()
        agents_path = parse_path_file(path_file_content)
    return dict(agents_path)

In [12]:
def gen_rate_list(alg:str, map_name:str, scen_name:str, actors_array:List[int], N=20, w=None):
    rate_list = []
    for num_actors in actors_array:
        rate, hl_expanded, ll_expanded = run(alg, map_name, scen_name, num_actors, N, w)
        print(f'Num actors: {num_actors}, Rate: {rate:.2f}, High level expanded: {hl_expanded:.2f}, Low level expanded: {ll_expanded:.2f}')
        rate_list.append((rate, hl_expanded, ll_expanded))
        if rate <= 0.1:
            break
    while len(rate_list) < len (actors_array):
        rate_list.append((0.0, 0, 0))
    
    return rate_list

In [4]:
rand_actors = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90]

In [6]:
# CBS random-32x32x10
rand_cbs = gen_rate_list('CBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors)
rand_cbs

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 121.75, Low level expanded: 185341.80


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 166.05, Low level expanded: 292156.05


  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
# ECBS random-32x32x10 w=1.5 more agents
rand_ecbs15 = gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen',rand_actors, w=1.5)
rand_ecbs15

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.25, Low level expanded: 2297.60


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 1.65, Low level expanded: 7141.25


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 80, Rate: 1.00, High level expanded: 22.90, Low level expanded: 203918.75


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 1.00, High level expanded: 27.00, Low level expanded: 239681.00


[(1.0, 1.25, 2297.6),
 (1.0, 1.65, 7141.25),
 (1.0, 4.05, 28271.85),
 (1.0, 5.7, 44017.1),
 (1.0, 8.85, 74103.8),
 (1.0, 12.1, 104986.05),
 (1.0, 15.05, 134369.5),
 (1.0, 18.95, 168669.15),
 (1.0, 22.9, 203918.75),
 (1.0, 27.0, 239681.0)]

In [8]:
# ECBS random-32x32x10 w=1.3 more agents
rand_ecbs13 = gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors, w=1.3)
rand_ecbs13

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.25, Low level expanded: 1972.40


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 1.65, Low level expanded: 5828.35


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 80, Rate: 1.00, High level expanded: 22.85, Low level expanded: 139537.10


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 1.00, High level expanded: 27.00, Low level expanded: 160758.00


[(1.0, 1.25, 1972.4),
 (1.0, 1.65, 5828.35),
 (1.0, 4.05, 21742.4),
 (1.0, 5.7, 32935.85),
 (1.0, 8.85, 53809.7),
 (1.0, 12.25, 76113.6),
 (1.0, 15.1, 94969.95),
 (1.0, 18.85, 117092.55),
 (1.0, 22.85, 139537.1),
 (1.0, 27.0, 160758.0)]

In [9]:
# ECBS random-32x32x10 w=1.1 more agents
rand_ecbs11 = gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors, w=1.1)
rand_ecbs11

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.25, Low level expanded: 1626.60


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 1.65, Low level expanded: 4411.90


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 80, Rate: 1.00, High level expanded: 28.55, Low level expanded: 92388.85


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 1.00, High level expanded: 33.00, Low level expanded: 101812.00


[(1.0, 1.25, 1626.6),
 (1.0, 1.65, 4411.9),
 (1.0, 4.05, 13602.95),
 (1.0, 5.95, 20245.1),
 (1.0, 9.6, 33060.5),
 (1.0, 14.15, 46718.2),
 (1.0, 17.25, 58086.3),
 (1.0, 22.95, 75856.25),
 (1.0, 28.55, 92388.85),
 (1.0, 33.0, 101812.0)]

In [15]:
maze_actors = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [26]:
# CBS maze
maze_cbs = gen_rate_list('CBS', 'maze-32-32-4.map', 'maze-32-32-4-even-1.scen', maze_actors)
maze_cbs

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.55, Low level expanded: 14727.35


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 5.90, Low level expanded: 91009.15


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 20, Rate: 0.70, High level expanded: 1313.79, Low level expanded: 19767258.50


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 30, Rate: 0.05, High level expanded: 3632.00, Low level expanded: 90649963.00


[(1.0, 1.55, 14727.35),
 (1.0, 5.9, 91009.15),
 (0.7, 1313.7857142857142, 19767258.5),
 (0.05, 3632.0, 90649963.0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0)]

In [16]:
# ECBS maze w=1.5
maze_ecbs15 = gen_rate_list('ECBS', 'maze-32-32-4.map', 'maze-32-32-4-even-1.scen', maze_actors, w=1.5)
maze_ecbs15

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.45, Low level expanded: 16079.40


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 2.80, Low level expanded: 54444.25


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 0.75, High level expanded: 188.87, Low level expanded: 6110303.20


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 100, Rate: 0.45, High level expanded: 241.67, Low level expanded: 7636940.11


[(1.0, 1.45, 16079.4),
 (1.0, 2.8, 54444.25),
 (1.0, 6.65, 154593.05),
 (1.0, 11.35, 285660.65),
 (1.0, 20.65, 558033.3),
 (1.0, 53.6, 1390517.35),
 (1.0, 90.2, 2360795.2),
 (0.8, 138.1875, 3985118.625),
 (0.75, 153.13333333333333, 4097545.0),
 (0.75, 188.86666666666667, 6110303.2),
 (0.45, 241.66666666666666, 7636940.111111111)]

In [24]:
# ECBS maze w=1.3
maze_ecbs13 = gen_rate_list('ECBS', 'maze-32-32-4.map', 'maze-32-32-4-even-1.scen', maze_actors, w=1.3)
maze_ecbs13

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.45, Low level expanded: 15200.30


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 2.80, Low level expanded: 48304.75


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 0.35, High level expanded: 392.00, Low level expanded: 10862948.57


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 100, Rate: 0.20, High level expanded: 445.75, Low level expanded: 9843569.50


[(1.0, 1.45, 15200.3),
 (1.0, 2.8, 48304.75),
 (1.0, 6.8, 137246.45),
 (1.0, 12.15, 276331.45),
 (1.0, 29.45, 717626.3),
 (0.9, 70.22222222222223, 1505417.5),
 (0.85, 113.0, 2384281.705882353),
 (0.9, 185.94444444444446, 4405942.611111111),
 (0.65, 234.46153846153845, 6000059.692307692),
 (0.35, 392.0, 10862948.57142857),
 (0.2, 445.75, 9843569.5)]

In [25]:
# ECBS maze w=1.1
maze_ecbs11 = gen_rate_list('ECBS', 'maze-32-32-4.map', 'maze-32-32-4-even-1.scen', maze_actors, w=1.1)
maze_ecbs11

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.45, Low level expanded: 12903.80


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 2.70, Low level expanded: 36903.55


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 80, Rate: 0.30, High level expanded: 623.00, Low level expanded: 12277785.33


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 0.05, High level expanded: 564.00, Low level expanded: 16271477.00


[(1.0, 1.45, 12903.8),
 (1.0, 2.7, 36903.55),
 (1.0, 10.0, 199532.3),
 (1.0, 16.3, 296106.35),
 (0.85, 36.23529411764706, 695702.6470588235),
 (0.8, 135.8125, 3612445.25),
 (0.7, 306.14285714285717, 5435383.714285715),
 (0.3, 187.83333333333334, 2934516.3333333335),
 (0.3, 623.0, 12277785.333333334),
 (0.05, 564.0, 16271477.0),
 (0.0, 0, 0)]

In [27]:
den_actors = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [28]:
# CBS den312d 
den_cbs = gen_rate_list('CBS', 'den312d.map', 'den312d-even-1.scen', den_actors)
den_cbs

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.40, Low level expanded: 16472.65


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 0.95, High level expanded: 4.37, Low level expanded: 131157.32


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 30, Rate: 0.25, High level expanded: 1065.60, Low level expanded: 39920976.40


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 40, Rate: 0.00, High level expanded: 0.00, Low level expanded: 0.00


[(1.0, 1.4, 16472.65),
 (0.95, 4.368421052631579, 131157.31578947368),
 (0.65, 322.2307692307692, 10552317.692307692),
 (0.25, 1065.6, 39920976.4),
 (0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0)]

In [29]:
# ECBS den312d w=1.5
den_ecbs15 = gen_rate_list('ECBS', 'den312d.map', 'den312d-even-1.scen', den_actors, w=1.5)
den_ecbs15

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.30, Low level expanded: 34266.15


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 2.50, Low level expanded: 147997.30


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 1.00, High level expanded: 53.20, Low level expanded: 5038232.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 100, Rate: 0.95, High level expanded: 52.95, Low level expanded: 4976657.89


[(1.0, 1.3, 34266.15),
 (1.0, 2.5, 147997.3),
 (1.0, 4.85, 419874.9),
 (1.0, 8.7, 793625.95),
 (1.0, 13.05, 1224775.5),
 (1.0, 18.2, 1688690.4),
 (1.0, 22.55, 2028463.0),
 (1.0, 28.7, 2634863.35),
 (1.0, 39.75, 3597905.8),
 (1.0, 53.2, 5038232.0),
 (0.95, 52.94736842105263, 4976657.894736842)]

In [30]:
# ECBS den312d w=1.3
den_ecbs13 = gen_rate_list('ECBS', 'den312d.map', 'den312d-even-1.scen', den_actors, w=1.3)
den_ecbs13

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.30, Low level expanded: 25913.05


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 2.50, Low level expanded: 111967.80


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 1.00, High level expanded: 61.90, Low level expanded: 4127844.95


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 100, Rate: 1.00, High level expanded: 53.60, Low level expanded: 3338927.55


[(1.0, 1.3, 25913.05),
 (1.0, 2.5, 111967.8),
 (1.0, 4.85, 286961.85),
 (1.0, 8.7, 528712.55),
 (1.0, 13.05, 808468.3),
 (1.0, 18.15, 1118123.6),
 (1.0, 23.8, 1466630.15),
 (1.0, 29.25, 1807911.05),
 (1.0, 39.5, 2441885.4),
 (1.0, 61.9, 4127844.95),
 (1.0, 53.6, 3338927.55)]

In [31]:
# ECBS den312d w=1.1
den_ecbs11 = gen_rate_list('ECBS', 'den312d.map', 'den312d-even-1.scen', den_actors, w=1.1)
den_ecbs11

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.30, Low level expanded: 15548.85


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 2.55, Low level expanded: 68857.50


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 0.75, High level expanded: 63.27, Low level expanded: 2378449.47


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 100, Rate: 0.75, High level expanded: 93.80, Low level expanded: 3583896.60


[(1.0, 1.3, 15548.85),
 (1.0, 2.55, 68857.5),
 (1.0, 4.95, 157298.3),
 (1.0, 9.1, 320128.25),
 (1.0, 14.6, 519549.45),
 (1.0, 23.05, 829349.2),
 (1.0, 51.65, 2047588.7),
 (1.0, 54.95, 2167707.6),
 (0.9, 51.0, 1951040.1666666667),
 (0.75, 63.266666666666666, 2378449.466666667),
 (0.75, 93.8, 3583896.6)]